In [2]:
import re
from newspaper import Article, ArticleException

def crawl_newspaper(url):
    """기사 링크를 입력으로 받아 제목, 내용 그리고 링크를 파싱하여 dictionary 형태로 반환하는 함수"""
    
    try:
        article = Article(url,
                          language='ko',
                          memorize_articles=False,
                          fetch_images=False,
                          skip_bad_cleaner=True)
        article.download()
        article.parse()

        article_data = {}
        article_data['title'] = article.title.strip()  # 기사 제목
        article_data['text'] = article.text.replace("\n", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
        article_data['url'] = url  # 기사 링크

        return article_data
    except ArticleException as AE:
        raise AE
    
    
tmp_article = crawl_newspaper("https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=022&aid=0003376441")

for k, v in tmp_article.items():
    print(k, ":", v[:100])
    
    
    
import requests
from bs4 import BeautifulSoup

def get_newspaper_list(query,  # 검색어
                       date=None,  # 기사 일자
                       start_page=1  # 뉴스 리스트 페이지
                      ):
    """네이버에서 검색어를 이용해 해당 기간 내의 뉴스 리스트를 반환하는 함수"""
    
    url = 'https://search.naver.com/search.naver?&where=news&query={query}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={start_date}&de={end_date}&start={start_page}'  # query와 기간을 조건으로 한 검색 url 
    if not date:
        date = datetime.now()  # 오늘 날짜 instance 생성
        date = date.strftime("%Y.%m.%d")  # 날자 instance를 문자열 형태로 변환 ex) 2019.06.30
    
    res = requests.get(url.format(query=query, 
                                  start_date=date, 
                                  end_date=date, 
                                  start_page=(start_page - 1) * 10 + 1))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.text, 'lxml')
    
    # 최대 기사 페이지 수를 확인
    pages = [tag.get_text() for tag in soup.select(".paging > a, .paging > strong")] # 이전페이지,1,2,3페이지
    max_page = 0
    for page in pages:
        try:
            page = int(page)
            max_page = max(page, max_page)
        except:
            continue
            
    if start_page > max_page:  # 더이상 페이지가 없을 경우 빈 리스트를 반환
        return []
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출
    link_list = [a['href'] for a in soup.select(".news > ul.type01 > li > dl > dt > a")]  
    return link_list    


from datetime import datetime, timedelta
from time import time

def crawl_newspaper_by_query(query,  # 검색어 키워드
                             start_date=None,
                             end_date=None,
                             max_page=10):
    
    
    article_list = []  # 기사 리스트
    crawled_url = set()  # 검색한 기사 중복 검색을 위한 변수
        
    if not start_date:
        start_date = datetime.now()  # 오늘 날짜 instance 생성
    else:
        start_date = datetime.strptime(start_date, "%Y.%m.%d")  # 2019.06.30과 같은 문자열을 python instance로 변경
        
    if not end_date:
        end_date = datetime.now()
    else:
        end_date = datetime.strptime(end_date, "%Y.%m.%d")
    
    while start_date < end_date:
        print("Date: ", end_date)
        for cur_page in range(1, max_page):
            print("Page: {}".format(cur_page))
            
            # end_date 날짜의 기사 링크 수집
            link_list = get_newspaper_list(query, end_date.strftime("%Y.%m.%d"), start_page=cur_page)

            if not link_list:  # 더이상 수집할 뉴스가 없을 경우
                break
            
            for link in link_list:
                # 중복 수집 방지
                if link in crawled_url:
                    continue
                else:
                    crawled_url.update([link])
                
                try:
                    article = crawl_newspaper(link)
                    article['date'] = start_date
                    article_list.append(article)
                except ArticleException as AE:
                    continue
        
        end_date = end_date - timedelta(days=1)  # 수집할 날짜를 하루 전으로 변경
    return article_list




title : 북한땅 10여m 올라간 트럼프… “군사분계선 넘은 것 영광” [남북미 정상, 사상 첫 판문점 회동]
text : 김정은 북한 국무위원장과 도널드 트럼프 미국 대통령이 30일 오후 3시47분 판문점 공동경비구역(JSA) 군사분계선 북측 지역으로 넘어간 뒤 다시 남측으로 내려오고 있다.판문점=연
url : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=022&aid=0003376441


In [3]:
start = time()
articles = crawl_newspaper_by_query("sk하이닉스 주가상승", "2019.06.01", "2019.07.15", max_page=10)
print(time() - start)
print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[0])



Date:  2019-07-15 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-14 00:00:00
Page: 1
Page: 2
Date:  2019-07-13 00:00:00
Page: 1
Page: 2
Date:  2019-07-12 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-11 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Date:  2019-07-10 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-09 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-08 00:00:00
Page: 1
Page: 2
Date:  2019-07-07 00:00:00
Page: 1
Page: 2
Date:  2019-07-06 00:00:00
Page: 1
Page: 2
Date:  2019-07-05 00:00:00
Page: 1
Page: 2
Date:  2019-07-04 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-03 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-02 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-01 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-06-30 00:00:00
Page: 1
Page: 2
Date:  2019-06-29 00:00:00
Page: 1
Page: 2
Date:  2019-06-28 00:00:00
Page: 1
Page: 2
Date:  2019-06-27 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-06-26 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-06-25 00:00:00
P

C:\Users\Administrator\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


NameError: name 'sentence' is not defined

In [4]:
from konlpy.tag import Okt

# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)

sentence = "SK텔레콤-LG전자, 5G로 클라우드와 로봇 결합했다… 로봇산업 혁신 기대"

words = tagger.pos(sentence, 
                   norm=True,  # 정규화 - 입니다 ㄹㄹ -> 입니다로 모아주는
                   stem=True)  # 어근화 - 원형으로 바꾸는 작업
for w, t in words:
    print(w, t)


from collections import Counter

words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set(["글자"])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..

for i, article in enumerate(articles):
    if i % 100 == 0:
        print(i)
        
    words = tagger.pos(article['text'], norm=True, stem=True)
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    
    
print(words_list[0])

vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    
    # 기사 내 서로 다른 단어 개수가 10개 이상인 기사만 사용
    if len(words) > 10:
        new_words_list.append((words, article))
        

        from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix.toarray()[:10, :20])


SK Alpha
텔레콤 Noun
- Punctuation
LG Alpha
전자 Noun
, Punctuation
5 Number
G Alpha
로 Noun
클라우드 Noun
와 Josa
로봇 Noun
결합 Noun
하다 Verb
… Punctuation
로봇산업 Noun
혁신 Noun
기대 Noun
0
100
200
300
(['뉴스', '이세영', '기자', '전자', 'SK', '하이닉스', '일본', '반도체', '소재', '대한', '수출', '규제', '직격탄', '주가', '상승', '반도체', '가격', '상승', '원인', '공급', '과잉', '현상', '해소', '가격', '안정', '오히려', '전자', 'SK', '하이닉스', '전화위복', '작용', '분석', '오후', '현재', '전자', '전날', '상승', '가격', '거래', '전자', '거래량', 'SK', '하이닉스', '역시', '전날', '대비', '오른', '거래', '중이', '거래량', '전자', 'SK', '하이닉스', '낸드', '플래시', '감산', '돌입', '추측', '가운데', '회사', '주가', '상승', '눈길', '사진', '연합뉴스', '최근', '세계', '메모리', '반도체', '업계', '돌발', '공급', '감소', '요인', '업황', '회복', '시점', '예상', '이야기', '삼성', '전자', '국내', '반도체', '업계', '실적', '부진', '둔화', '전망', '심리', '주가', '반영', '분석', '김경민', '하나', '금융투자', '연구원', '낸드', 'Negative', 'AND', '감산', '공급', '과잉', '개선', '업황', '통과', '대감', '주가', '상승', '견인', '최근', '발생', '도시바', '메모리', 'TMC', '요카이', '공장', '정전', '일본', '수출', '규제', '강화', '재고', '소진', '기회', '제공', '분석', '또한', '향후', '월간', '신

In [5]:
from sklearn.decomposition import NMF

K = 10  # topic 개수
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (392, 10)
H shape: (10, 4303)


In [6]:
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
        
    print("\n", "="*20)

0th topic
반도체 일본 수출 규제 소재 전자 생산 한국 주가 업체 국내 가격 조치 기업 정부 영향 하이닉스 오히려 지난 차질 
1th topic
거래 중이 거래량 백만원 기록 주가 전일 금은 고가 저가 대비 하락 상승 주식 대금 가격 각각 수준 종목 LG 
2th topic
종가 등락 SK 하이닉스 차트 기준 흐름 공감 최저 기록 주가 현재 최고 종목 대비 분석 주식 지난 변화 최근 
3th topic
KODEX 기사 금융 전자 삼성 투자자 레버리지 LG SK 이중 동생 공동 AI 경로 중점 알고리즘 전문 실시간 작성 신문 
4th topic
증가 사업 성장 확대 매출 출시 배당 실적 편입 부각 개선 신규 매력 본격 보유 하반기 데이터 해소 이익 전망 
5th topic
포인트 지수 오른 코스피 거래 상승 마감 인하 기관 미국 셀트리온 금리 출발 외국인 매도 증시 반면 전장 증권 전날 
6th topic
순이익 업종 기록 반도체 시가총액 분석 장비 영업 매출 이익 종목 외국인 식수 테크 주의 분류 순위 소진 리포트 주로 
7th topic
수익률 위험 변동성 업종 월간 비교 대비 가장 기록 의미 때문 표준편차 지수 브레인 동기간 수준 전자 탄력 유지 초과 
8th topic
투자 목표 의견 낸드 수요 SK 연구원 이익 실적 영업 하이닉스 예상 증권 분기 서버 회복 감소 주가 유지 전망 
9th topic
개인 종목 매수 외국인 상위 수익률 평균 투자자 매도 기간 주가 상반기 반면 성과 제외 코스피 모두 개미 올해 아난 


In [7]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:3]:
        print(words_list[index][1]['title'])
    print("\n", "="*20)

0th topic
일본 수출 규제, 타격 ‘물음표’…소재 업체 주가 강세
日 무역전쟁 휘말린 국내 증시···투자활력 잃을까
WSJ “日 반도체 수출 제재, 오히려 삼성전자에 호재”

1th topic
11일, 외국인 거래소에서 SK하이닉스(+0.6%), KB금융(-1%) 등 순매도
美 금리인하 기대감에 코스피 반등…2072포인트 마감
코스피, 외국인 '팔자'에 2060선 마감···2069.11

2th topic
[코스피 시가총액 Top 10 :장중시황] 삼성전자 주가 0.11% 상승, SK하이닉스 0.75% 상승 , 현대차 -0.35% 하락, 삼성전자우, 셀트리온 주가 순으로  시가총액 상위 기록.
(10일주식장마감동향)SK하이닉스 72900원 마감
(2일마감증시)SK하이닉스 71400원 마감

3th topic
外人 韓증시서 2조원 넘게 순매도…가장 많이 팔아치운 종목은
[책갈피] 투자 대가들의 가치평가 활용법 ⑦
28일, 외국인 거래소에서 삼성전자(+1.08%), LG화학(+1.43%) 등 순매수

4th topic
코스피, 美FOMC 통화완화 기조에도 `무덤덤`…강보합 마감
18일, 거래소 기관 순매수상위에 통신업 업종 2종목
글로벌 양적완화 물결…코스피 나흘 만에 2100선 회복

5th topic
코스피, 美 금리인하 기대감 선반영돼 약보합 출발
[오늘의 추천종목-한국투자증권]SK텔레콤·현대차  등
[특징주] 시스웍 상승 종목 4위 등극.... 금일 주가 3395원, 전 영업일 대비 535원 상승...

6th topic
[특징주] 램테크놀러지 상승 종목 7위 등극.... 금일 주가 6320원, 전 영업일 대비 740원 상승...
[장마감] 램테크놀러지 종목 상승 5 위 종목 차지해... 시가총액순위 코스닥 810위, 종목 현재 거래량531만683주
[오늘의 추천종목-한국투자증권]롯데지주·세아제강  등

7th topic
[ET투자뉴스]삼성전자_상장주식수 대비 거래량은 0.15%로 적정수준
SK하이닉스, DRAM·NAND 업황 개선...“주가상승 흐

In [8]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지

W shape: (392, 10)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 392 samples in 0.000s...
[t-SNE] Computed neighbors for 392 samples in 0.015s...
[t-SNE] Computed conditional probabilities for sample 392 / 392
[t-SNE] Mean sigma: 0.063486
[t-SNE] KL divergence after 250 iterations with early exaggeration: 53.538349
[t-SNE] KL divergence after 1000 iterations: 0.191270


In [9]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [10]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})

p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [15]:
query = '일본'
query_index = word2id[query]  # 상승관련 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 SKT 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
ranks = query_vec.toarray().flatten().argsort()[::-1]

for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])

[특징주] 삼성전자·SK하이닉스, 일본 수출 규제 여파로 ‘동반 하락’ http://www.fntimes.com/html/view.php?ud=2019070809243869896c0eb6f11e_18
일본 수출 규제에 반도체주 ‘긴장’· 불매운동 수혜주 ‘상승’ http://www.ekn.kr/news/article.html?no=441784
증시까지 번진 日 불매운동… 수혜주·피해주는? http://www.dt.co.kr/contents.html?article_no=2019070802109932048013&ref=naver
아사히 대신 하이트, 日 불매운동 확산에 ‘애국테마株’ 날개 http://www.dt.co.kr/contents.html?article_no=2019070902101132048003&ref=naver
일본 수출 규제, 타격 ‘물음표’…소재 업체 주가 강세 http://www.economytalk.kr/news/articleView.html?idxno=190202
日 보복 규제에 희비 엇갈리는 업종 주가 http://www.fnnews.com/news/201907101601009300
반도체 대장주 반등…일본 보복 불구 외인 쓸어담았다 http://www.economytalk.kr/news/articleView.html?idxno=191315
“일본 수출규제 파문, 한일 양국 기업들이 해결사” http://www.etoday.co.kr/news/section/newsview.php?idxno=1773650
日 무역전쟁 휘말린 국내 증시···투자활력 잃을까 http://www.sisajournal-e.com/news/articleView.html?idxno=202668
아베의 자충수…韓 반도체보다 日 소재주 더 빠졌다 http://news.mk.co.kr/newsRead.php?year=2019&no=519744
